### 设置RNN的参数

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
tf.set_random_seed(1) #set random seed

#导入数据
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

#hyperparameters
lr = 0.001      #leaning rate
training_iters = 100000    #train step上限
batch_size = 128    #每次读取的数据个数
n_inputs = 28    #MNIST data input 读入一行
n_steps = 28     #time steps    和n_inputs组合一起为 一张图片的大小(28*28)
n_hidden_utils = 128    #每个cell 中的神经元个数
n_classes = 10    #MNIST class

/home/lhw/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
# x y placeholder
x = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.float32, [None, n_classes])

#对weights biases 初始值的定义
weights = {
    #shape (28,128)
    'in': tf.Variable(tf.random_normal([n_inputs, n_hidden_utils])),
    #shape (128, 10)
    'out': tf.Variable(tf.random_normal([n_hidden_utils, n_classes]))
}
biases = {
    #shape (128, )
    'in': tf.Variable(tf.constant(0.1, shape=[n_hidden_utils, ])),
    #shape (10, )
    'out': tf.Variable(tf.constant(0.1, shape=[n_classes, ]))
}

### 定义RNN的主体结构

In [3]:
def RNN(X, weights, biases):
    # (128batchs, 28 steps, 29 inputs)X ==> (128 batches * 28 steps, 28 inputs) 分离出行
    X = tf.reshape(X, [-1, n_inputs])
    # X_in = W*X + b   #affine 同时扩充行维度
    X_in = tf.matmul(X, weights['in']) + biases['in']
    # X_in ==> (128 batches, 28 steps, 128 hidden)还原成原来的形式
    X_in = tf.reshape(X_in, [-1, n_steps, n_hidden_utils])
    
    #使用basic LSTM Cell
    lstm_cell = tf.contrib.rnn.BasicLSTMCell(n_hidden_utils, forget_bias=1.0)
    init_state = lstm_cell.zero_state(batch_size, dtype=tf.float32)#初始化全为0
    #with tf.variable_scope('train'): 
    outputs, final_state = tf.nn.dynamic_rnn(lstm_cell, X_in, initial_state=init_state, time_major=False)
    results = tf.matmul(final_state[1], weights['out']) + biases['out']
    
    return results

### 训练RNN

In [4]:
pred = RNN(x, weights, biases)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
train_op = tf.train.AdamOptimizer(lr).minimize(cost)

correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))#tf.argmax(vector, 1)：返回的是vector中的最大值的索引号
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    step = 0
    while step * batch_size < training_iters:
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = batch_xs.reshape([batch_size, n_steps, n_inputs])
        sess.run([train_op], feed_dict={
            x:batch_xs,
            y:batch_ys
        })
        if step % 20 == 0:
            print(sess.run(accuracy, feed_dict={
                x:batch_xs,
                y:batch_ys
            }))
        step += 1

0.265625
0.703125
0.796875
0.867188
0.867188
0.882812
0.875
0.882812
0.898438
0.914062
0.867188
0.929688
0.914062
0.9375
0.90625
0.929688
0.945312
0.953125
0.929688
0.96875
0.945312
0.984375
0.90625
0.976562
0.96875
0.953125
0.96875
0.945312
0.960938
0.945312
0.96875
0.96875
0.953125
0.992188
0.976562
0.960938
0.953125
0.945312
0.953125
0.960938
